In [ ]:
%%bash
install -d ~/.local/src/smartabyar-smartvillage
git clone https://github.com/computate-org/computate.git ~/computate
git clone https://github.com/computate-org/computate-search.git ~/computate-search
git clone https://github.com/computate-org/computate-vertx.git ~/computate-vertx
git clone https://github.com/computate-org/smartvillage-platform.git ~/smartvillage-platform
git clone https://github.com/computate-org/smartvillage-platform-static.git ~/smartvillage-platform-static
git clone https://github.com/computate-org/smartabyar-smartvillage.git ~/smartabyar-smartvillage
git clone https://github.com/computate-org/smartabyar-smartvillage-static.git ~/smartabyar-smartvillage-static
echo DONE

In [ ]:
%%bash
install -d ~/.ansible/roles
git clone git@github.com:computate-org/computate_project.git ~/.ansible/roles/computate.computate_project
echo DONE

In [ ]:
%%bash
#ansible-playbook ~/.ansible/roles/computate.computate_project/install.yml -e SITE_NAME=computate-search -e ENABLE_CODE_GENERATION_SERVICE=true -e SYSTEMD_ENABLED=false -e SITE_PREFIX=/opt/app-root/src
(cd ~/computate && env JAVA_HOME=/usr/lib/jvm/java-11-openjdk mvn clean install)
echo DONE

In [ ]:
%%bash
ansible-playbook ~/.ansible/roles/computate.computate_project/install.yml \
  -e SITE_NAME=computate-search \
  -e SYSTEMD_ENABLED=false \
  -e SITE_PREFIX=/opt/app-root/src \
  -e SOLR_HOST_NAME=solr \
  -e SOLR_PORT=8983 \
  -e SOLR_SSL=false \
  -e SOLR_URL="http://solr:8983/solr/computate-search" \
  -e SOLR_URL_COMPUTATE="http://solr:8983/solr/computate" \
  -e POSTGRES_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e ZOOKEEPER_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SOLR_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace)
echo DONE

In [ ]:
%%bash
env SITE_NAME=computate-search \
  SITE_PATH=$HOME/computate-search \
  COMPUTATE_SRC=$HOME/computate \
  SITE_LANG=enUS \
  ~/computate/bin/enUS/index.sh
echo DONE

In [ ]:
%%bash
ansible-playbook ~/.ansible/roles/computate.computate_project/install.yml \
  -e SITE_NAME=computate-vertx \
  -e SYSTEMD_ENABLED=false \
  -e SITE_PREFIX=/opt/app-root/src \
  -e SOLR_HOST_NAME=solr \
  -e SOLR_PORT=8983 \
  -e SOLR_SSL=false \
  -e SOLR_URL="http://solr:8983/solr/computate-vertx" \
  -e SOLR_URL_COMPUTATE="http://solr:8983/solr/computate" \
  -e POSTGRES_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e ZOOKEEPER_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SOLR_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace)
echo DONE

In [ ]:
%%bash
env SITE_NAME=computate-vertx \
  SITE_PATH=$HOME/computate-vertx \
  COMPUTATE_SRC=$HOME/computate \
  SITE_LANG=enUS \
  ~/computate/bin/enUS/index.sh
echo DONE

In [ ]:
%%bash
ansible-playbook ~/.ansible/roles/computate.computate_project/install.yml \
  -e SITE_NAME=smartvillage-platform \
  -e SYSTEMD_ENABLED=false \
  -e SITE_PREFIX=/opt/app-root/src \
  -e SOLR_HOST_NAME=solr \
  -e SOLR_PORT=8983 \
  -e SOLR_SSL=false \
  -e SOLR_URL="http://solr:8983/solr/smartvillage-platform" \
  -e SOLR_URL_COMPUTATE="http://solr:8983/solr/computate" \
  -e POSTGRES_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e ZOOKEEPER_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SOLR_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace)
echo DONE

In [ ]:
%%bash
env SITE_NAME=smartvillage-platform \
  SITE_PATH=$HOME/smartvillage-platform \
  COMPUTATE_SRC=$HOME/computate \
  SITE_LANG=enUS \
  ~/computate/bin/enUS/index.sh
echo DONE

In [ ]:
%%bash
SITE_HOST_NAME="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get route/sumodev -o jsonpath={.spec.host})"
AUTH_SECRET_NAME="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get secret/keycloak-client-secret-smartvillage -o jsonpath={.data.CLIENT_SECRET})"
AUTH_CLIENT="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get secret/keycloak-client-secret-smartvillage -o jsonpath={.data.CLIENT_ID} | base64 -d)"
AUTH_SECRET="$(oc -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) get secret/keycloak-client-secret-smartvillage -o jsonpath={.data.CLIENT_SECRET} | base64 -d)"

oc extract -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) secret/smartvillage-kafka --to=$HOME/smartabyar-smartvillage/config/ --keys=user.p12 --confirm
KAFKA_SSL_KEYSTORE_PASSWORD="$(oc get -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) secret/smartvillage-kafka -o 'jsonpath={.data.user\.password}' | base64 -d)"
oc extract -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) secret/default-cluster-ca-cert --to=$HOME/smartabyar-smartvillage/config/ --keys=ca.p12 --confirm
KAFKA_SSL_TRUSTSTORE_PASSWORD="$(oc get -n $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) secret/default-cluster-ca-cert -o 'jsonpath={.data.ca\.password}' | base64 -d)"

ansible-playbook ~/.ansible/roles/computate.computate_project/install.yml \
  -e SITE_NAME=smartabyar-smartvillage \
  -e SYSTEMD_ENABLED=false \
  -e SITE_PREFIX=/opt/app-root/src \
  -e SOLR_HOST_NAME_COMPUTATE=solr \
  -e SOLR_PORT_COMPUTATE=8983 \
  -e SOLR_SSL_COMPUTATE=false \
  -e SOLR_HOST_NAME=solr \
  -e SOLR_PORT=8983 \
  -e SOLR_SSL=false \
  -e SOLR_URL="http://solr:8983/solr/smartabyar-smartvillage" \
  -e SOLR_URL_COMPUTATE="http://solr:8983/solr/computate" \
  -e ZOOKEEPER_HOST_NAME=zookeeper \
  -e ZOOKEEPER_PORT=2181 \
  -e JDBC_HOST=postgres-primary \
  -e JDBC_PORT=5432 \
  -e POSTGRES_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e ZOOKEEPER_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SOLR_NAMESPACE=$(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace) \
  -e SITE_BASE_URL="https://${SITE_HOST_NAME}" \
  -e STATIC_BASE_URL="https://${SITE_HOST_NAME}/static" \
  -e AUTH_CLIENT="${AUTH_CLIENT}" \
  -e AUTH_SECRET="${AUTH_SECRET}" \
  -e KAFKA_SSL_KEYSTORE_PASSWORD="${KAFKA_SSL_KEYSTORE_PASSWORD}" \
  -e KAFKA_SSL_TRUSTSTORE_PASSWORD="${KAFKA_SSL_TRUSTSTORE_PASSWORD}" \
  -e KAFKA_BROKERS="default-kafka-bootstrap:9093" \
  -e ENABLE_IMPORT_DATA=false
echo DONE

In [ ]:
%%bash
env SITE_NAME=smartabyar-smartvillage \
  SITE_PATH=$HOME/smartabyar-smartvillage \
  COMPUTATE_SRC=$HOME/computate \
  SITE_LANG=enUS \
  ~/computate/bin/enUS/generate.sh
echo DONE

In [ ]:
%%bash
echo '
kind: Service
apiVersion: v1
metadata:
  name: sumodev
  namespace: smart-village-faeeb6c
spec:
  ports:
    - protocol: TCP
      port: 12080
      targetPort: 12080
  type: ClusterIP
  selector:
    notebook-name: sumo
' | oc apply -f -
